In [ ]:
train_data = pd.read_table('./data/ratings_train.txt') 
train_data.dropna(inplace=True)
train_data.head()

In [ ]:
def clean_txt(doc):
    doc = re.sub('[^ㄱ-ㅎ가-힣 ]', ' ', doc) # 한글이랑 공백(spacebar)만 남기고
    doc = re.sub('\s+', ' ', doc) # 다중 공백 하나의 공백으로 변환
    doc = np.nan if doc == '' or doc == ' ' else doc # 전처리 과정에서 공백만 남는경우 삭제
    return doc

train_data['clean_doc'] = train_data['document'].apply(clean_txt)
train_data.dropna(inplace=True)
train_data.head()

In [ ]:
stw = pd.read_csv('./data/stopwords-ko.txt', header=None) # 불용어 사전(?) 활용
stw_list = list(stw[0].values)
tokenized_data = []
for sen in train_data['clean_doc']:
    temp_x = mecab.morphs(sen)
    temp_x = [word for word in temp_x if not word in stw_list]
    tokenized_data.append(temp_x)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

sentences = [
    'nice great best amazing', # 1
    'stop lies', # 0
    'pitiful nerd', # 0
    'excellent work', # 1
    'supreme quality', # 1
    'bad', # 0
    'highly respectable' # 1
]
y_train = [1,0,0,1,1,0,1] # 라벨도 부여해보자

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
t = Tokenizer()
t.fit_on_texts(sentences)  # tokenizer를 sentences에 있는 단어에 대하여 fit
vocab_size = len(t.word_index)+1 # zero_padding 떄문에 +1 여분 자리
vocab_size

In [ ]:
# 사전에 훈련된 Word2vec 임베딩 가져오기

import gensim.downloader as api
path = api.load('word2vec-google-news-300', return_path=True)
path

from gensim.models import KeyedVectors
w2v_model = KeyedVectors.load_word2vec_format(path, binary=True)

In [ ]:
embedding_matrix = np.zeros((vocab_size, 300))
def get_vector(word):
    if word in w2v_model:
        return w2v_model[word]
    else:
        return None
for word, index in t.word_index.items():
    # 단어와 맵핑되는 사전 훈련된 임베딩 벡터값
    vector_value = get_vector(word)
    if vector_value is not None:
        embedding_matrix[index] = vector_value

In [ ]:
# 사전 훈련된 모델 사용하기

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten, Input

model = Sequential()
model.add(Input(shape=(max_len,), dtype='int32'))
e = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_len, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=100, verbose=2)